In [1]:
import sys, os
from collections import defaultdict
from rdflib import Graph, Literal, URIRef


sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
from cimple_querying import localGraph

cimple = localGraph('KGs_v6/')

uris_in_namespaces, unique_uris_in_namespaces = cimple.get_full_namespace_dict()

In [2]:
type_statements = cimple.get_all_triples_from_predicate(URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
predicates_per_types = defaultdict(dict)
for i in set(type_statements.values()):
    predicates_per_types[i]['as_subjects']=cimple.get_unique_predicates_for_recordType(i, as_subject=True)
    predicates_per_types[i]['as_objects']=cimple.get_unique_predicates_for_recordType(i, as_subject=False)

In [3]:
predicates_per_types.keys()

dict_keys([rdflib.term.URIRef('http://schema.org/Rating'), rdflib.term.URIRef('http://schema.org/ClaimReview'), rdflib.term.URIRef('http://schema.org/Review'), rdflib.term.URIRef('http://schema.org/SocialMediaPosting'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology'), rdflib.term.URIRef('http://schema.org/Organization'), rdflib.term.URIRef('http://schema.org/NewsArticle'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'), rdflib.term.URIRef('http://schema.org/Claim'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept')])

## TWITTS

In [4]:
predicates_per_types[URIRef('http://schema.org/SocialMediaPosting')]

{'as_subjects': {rdflib.term.URIRef('http://data.cimple.eu/ontology#hasEmotion'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#hasPoliticalLeaning'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#hasSentiment'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#mentionsConspiracy'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#promotesConspiracy'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#readability_score'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#related'),
  rdflib.term.URIRef('http://schema.org/dateCreated'),
  rdflib.term.URIRef('http://schema.org/mentions'),
  rdflib.term.URIRef('http://schema.org/text'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')},
 'as_objects': {rdflib.term.URIRef('http://schema.org/itemReviewed')}}

In [22]:
twitts = cimple.get_all_triples_from_predicate_subjectobject(URIRef('http://schema.org/text'), URIRef('http://schema.org/SocialMediaPosting'), subject=True)
twitter_data = dict()
for uri in twitts.keys():
    twitter_data[str(uri).rsplit('/', 1)[-1]]=str(twitts[uri])

In [23]:
missing_c=0
for text in twitter_data.values():
    if len(text)==0:
        missing_c+=1
print(f"Rate of missing data is {missing_c/len(twitter_data)} out of {len(twitter_data)} total documents")

Rate of missing data is 0.010993212885957365 out of 10461 total documents


## AFP

In [24]:
predicates_per_types[URIRef('http://schema.org/NewsArticle')]

{'as_subjects': {rdflib.term.URIRef('http://data.cimple.eu/ontology#hasEmotion'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#hasPoliticalLeaning'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#hasSentiment'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#mentionsConspiracy'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#promotesConspiracy'),
  rdflib.term.URIRef('http://data.cimple.eu/ontology#readability_score'),
  rdflib.term.URIRef('http://schema.org/articleBody'),
  rdflib.term.URIRef('http://schema.org/author'),
  rdflib.term.URIRef('http://schema.org/dateCreated'),
  rdflib.term.URIRef('http://schema.org/headline'),
  rdflib.term.URIRef('http://schema.org/mentions'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')},
 'as_objects': set()}

In [25]:
afp = cimple.get_all_triples_from_predicate_subjectobject(URIRef('http://schema.org/articleBody'), URIRef('http://schema.org/NewsArticle'), subject=True)
afp_data = dict()
for uri in afp.keys():
    afp_data[str(uri).rsplit('/', 1)[-1]]=str(afp[uri])

In [26]:
missing_c=0
for text in afp_data.values():
    if len(text)==0:
        missing_c+=1
print(f"Rate of missing data is {missing_c/len(afp_data)} out of {len(afp_data)} total documents")

Rate of missing data is 0.0 out of 193932 total documents
